# Lab 2 - Spark SQL
## This Lab will show you how to work with Spark SQL


### Spark SQL
This notebook guides you through querying data with Apache Spark, including how to create and use DataFrames, run SQL queries, apply functions to the results of SQL queries, join data from different data sources, and visualize data in graphs.

This notebook uses pySpark, the Python API for Spark. Some knowledge of Python is recommended. This notebook runs on Python 2 with Spark 1.6 and 2.0.

If you are new to Apache Spark, see the first module in this series: __Introduction to Apache Spark: Basic Concepts__.

Before you can run SQL queries on data in an Apache Spark environment, you need to enable SQL processing and then move the data to the structured format of a DataFrame.

### Enable SQL processing
The way you enable SQL processing with Spark 1.6 is to create an sqlContext. With Spark 2.0, the preferred method is to use the new sparkSession object, but the sqlContext object is still supported. 

Use the predefined Spark Context, `sc`, which contains the connection information for Spark, to create a sqlContext:

# Step 1

<h3>Getting started: Create a SQL Context</h3>

<b>Type:</b>


from pyspark.sql import SQLContext<br>
sqlContext = SQLContext(sc)


In [1]:
#Create the SQLContext below:


# Step 2

<h3>Dowload a JSON Recordset to work with</h3>

Let's download the data, we can run commands on the console of the server (or docker image) that the notebook enviroment is using. To do so we simply put a "!" in front of the command that we want to run. For example:

!pwd

To get the data we will download a file to the enviroment. Simple run these two commands, the first just ensures that the file is removed if it exists:

!rm world_bank.json.gz -f <br>
!wget https://raw.githubusercontent.com/bradenrc/sparksql_pot/master/world_bank.json.gz

In [1]:
#enter the commands to remove and download file here


# Step 3
<h3>Create a Dataframe</h3>

Now you can create the Dataframe, note that if you wanted to see where you downloaded the file you can run !pwd or !ls

Instead of creating an RDD to read the file, you'll create a Spark DataFrame. Unlike an RDD, a DataFrame creates a schema around the data, which supplies the necessary structure for SQL queries. A self-describing format like JSON is ideal for DataFrames, but many other file types are supported, including text (CSV) and Parquet.

To create the Dataframe type:

example1_df = sqlContext.read.json("./world_bank.json.gz")


In [4]:
#create the Dataframe here:


<h3>We can look at the schema:</h3>

Type or copy/paste the following into the cell below:<br>
example1_df.printSchema()

In [2]:
#print out the schema


<h3>Dataframes work like RDDs, you can map, reduce, groupby, etc. 
<br>Take a look at the first two rows of data using "take(2)". <br></h3>
Type of copy/paste the following into the cell below:<br>
example1_df.take(2)</h3>

In [5]:
#Use take on the dataframe to pull out 2 rows


#### For increased readability, run the cell below to include a row of asterisks in between the data rows:<br>

In [ ]:
for row in example1_df.take(2):
    print row
    print "*" * 20

# Step 4 
<h3>Register a table</h3>

Using
DataframeName.registerTempTable("name_of_table")

Create a table named "world_bank"

In [9]:
#Create the table to be referenced via SQL


# Step 5
<h3>Running SQL Statements</h3>
Using SQL, get the first 2 records from the table defined in the cell above (This is done using the SQL "limit 2" clause).<br>
#### Hint: sqlContext.sql("SQL Statement") will execute the SQL statement between double quotes and return a Dataframe with the records corresponding to the result of the query.

In [1]:
#Use SQL to select from table limit 2 and print the output


#### The method show(n) shows the first n elements of a dataframe. Use it below to display the rows resulting from your SQL query.

In [ ]:
# Show the content of the dataframe resulting from your SQL query.


#### Notice that the data produced by show() is not very nicely formatted... We can take care of this by converting the Spark dataframe to a Pandas dataframe.

In [2]:
# Extra credit, take the Dataframe you created with the two records and convert it into Pandas. This is done by applying the .toPandas() method 
# to the SparkSQL dataframe.


#### <span style="color:red">Warning: Converting a Spark dataframe to Pandas effectively executes a "collect" on the Spark dataframe. What is a potential problem with this?</span>

#### You can also run the following cell to select all columns from the table and print information about the resulting DataFrame and schema of the data: (assuming you named your table world_bank). Notice that printing a dataframe does not print its "contents". Remember that you need to use .show() to display the contents of a Spark dataframe.

In [ ]:
temp_df =  sqlContext.sql("select * from world_bank")

print type(temp_df)
print "*" * 20
print temp_df

### <span style="color:red">Review of Spark concepts</span>
You should notice that executing:<br><span style="color:blue">sqlContext.sql("SQL statement")</span><span style="color:black"><br> with any SQL statement runs almost immediately, but<br><span style="color:blue">sqlContext.sql("SQL statement").count()</span><span style="color:black"> should take longer to execute.

In [2]:
# Try the test above in this cell.


### <span style="color:red"> Can you tell why this is happening?</span>

### Experiment with an aggregation in SparkSQL

In [7]:
#Now using the same "world_bank" table, let's produce a simple count based on grouping over a column, for example "regionname"


In [3]:
# With JSON data you can reference the nested data
# If you look at Schema above you can see that Sector.Name is a nested column
# Select that column and limit to reasonable output (like 2)

#sqlContext.sql("select id, sector.Name from world_bank limit 2").collect()

# Step 6

<h3>Creating simple graphs</h3>
Using Pandas and the matplotlib library, we can create some simple visualizations.

### First create a SQL statement that returns a resonable number of elements.
For example, we can count the number of projects (rows) by countryname. But before that, let's go through a quick setup...

In [3]:
# we need to tell the charting library (matplotlib) to display charts inline
# just run this cell
%matplotlib inline 
import matplotlib.pyplot as plt, numpy as np

#### Now we can write simple sql to look at some data. Remember to add .toPandas() for nicer formatting. A slightly different option than what we've used so far is to create a variable and set it to the SQL statement. The cell below gives you some help with commented code.

In [ ]:
# query = "select count(*) as Count, countryname from world_bank group by countryname"
# chart1_df = sqlContext.sql(query).toPandas()
# print chart1_df

#### Now using the matplotlib .plot method, we can then plot data from the previous Pandas dataframe...

In [23]:
# now take the variable (or same sql statement) and use the method:
# .plot(kind='bar', x='countryname', y='Count', figsize=(12, 5))


### <span style="color:blue">The chart produced above is very "busy" on the x-axis. Can you produce a less busy chart?</span>

# Step 7

<h3>Creating a dataframe "manually" by adding a schema to an RDD</h3>

#### First, we need to create an RDD of pairs or triplets. This can be done using some code (such as a for loop) as seen in the instructor's example, or more simply by assigning values to an array.

In [ ]:
# Default array defined below. Feel free to change as desired.
array=[[1,1,1],[2,2,2],[3,3,3],[4,4,4],[5,5,5]]
my_rdd = sc.parallelize(array)
my_rdd.collect()

In the cell below this one, create your dataframe manually, following these steps:<br>
1- Define your schema columns as a string<br>
2- Build the schema object using StructField<br>
3- Apply the schema object to the RDD<br>

### Note: The cell below is missing some code and will not run properly until the missing code has been completed.

In [ ]:
from pyspark.sql.types import *

# The schema is encoded in a string. Complete the string below
schemaString = ""

# MissingType() should be either StringType() or IntegerType(). Please replace as required.
fields = [StructField(field_name, MissingType(), True) for field_name in schemaString.split()]
schema = StructType(fields)

# Apply the schema to the RDD.
schemaExample = sqlContext.createDataFrame(use_your_rdd_name_here, schema)

# Register the DataFrame as a table. Add table name below as parameter to registerTempTable.
schemaExample.registerTempTable("")


In [ ]:
# Run some select statements on your newly created DataFrame and display the output


# Step 8

## Loading data from a dashDB datasource

### In a different browser tab, create a dashDB service, add credentials and come back to this notebook. <br>Each dashDB instance in bluemix is created with a "GOSALES" set of tables which we can reuse for the purpose of this example. (You can create your own table if you wish...)<br><br>Replace the Xs in the cell below with proper credentials and verify access to dashDB tables.

In [ ]:
salesDF = sqlContext.read.format('jdbc').\
          options(url='jdbc:db2://dashdb-entry-yp-xxxxx.services.dal.bluemix.net:50000/BLUDB:user=xxxxx;password=xxxxxx;',\
                  dbtable='GOSALES.BRANCH').load()

In [ ]:
salesDF.show(3)